In [1]:
import matplotlib.pyplot as plt
import seaborn as sb
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchmetrics as tm
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
with open('datasets/activity_labels.txt') as f:
	defined_labels = [line.strip().split(" ")[1] for line in f]

In [3]:
X_train = []
with open("datasets/Train/X_train.txt") as f:
	for line in f:
		arr = line.strip().split(" ")
		arr = [float(num) for num in arr]
		X_train.append(torch.tensor(arr))
y_train = []
with open("datasets/Train/y_train.txt") as f:
	for line in f:
		y_train.append(int(line.strip())-1)

X_test = []
with open("datasets/Test/X_test.txt") as f:
	for line in f:
		arr = line.strip().split(" ")
		arr = [float(num) for num in arr]
		X_test.append(torch.tensor(arr))
y_test = []
with open("datasets/Test/y_test.txt") as f:
	for line in f:
		y_test.append(int(line.strip())-1)

In [4]:
class TabularDataset(Dataset):
	def __init__(self, ts, labels):
		self.ts = ts
		self.labels = labels

	def __len__(self):
		return len(self.labels)

	def __getitem__(self, idx):
		datas = self.ts[idx]
		label = self.labels[idx]
		return datas, label

In [5]:
train_data = TabularDataset(X_train, y_train)
test_data = TabularDataset(X_test, y_test)

In [6]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
class ClassificationMetrics(nn.Module):
    def __init__(self, num_classes: int, mode: str):
        super().__init__()
        self.num_classes = num_classes
        self.mode = mode

        self.categ_pc = tm.Precision(num_classes=self.num_classes, average="none")
        self.macro_pc = tm.Precision(num_classes=self.num_classes, average="macro")
        self.micro_pc = tm.Precision(num_classes=self.num_classes, average="micro")
        self.weigh_pc = tm.Precision(num_classes=self.num_classes, average="weighted")

        self.categ_rc = tm.Recall(num_classes=self.num_classes, average="none")
        self.macro_rc = tm.Recall(num_classes=self.num_classes, average="macro")
        self.micro_rc = tm.Recall(num_classes=self.num_classes, average="micro")
        self.weigh_rc = tm.Recall(num_classes=self.num_classes, average="weighted")

        self.categ_f1 = tm.F1(num_classes=self.num_classes, average="none")
        self.macro_f1 = tm.F1(num_classes=self.num_classes, average="macro")
        self.micro_f1 = tm.F1(num_classes=self.num_classes, average="micro")
        self.weigh_f1 = tm.F1(num_classes=self.num_classes, average="weighted")

        self.cnfs_mat = tm.ConfusionMatrix(num_classes=self.num_classes, normalize="true")

        self.mcc = tm.MatthewsCorrcoef(num_classes=self.num_classes)

    def to(self, device: torch.device):
        self.categ_pc.to(device)
        self.macro_pc.to(device)
        self.micro_pc.to(device)
        self.weigh_pc.to(device)

        self.categ_rc.to(device)
        self.macro_rc.to(device)
        self.micro_rc.to(device)
        self.weigh_rc.to(device)

        self.categ_f1.to(device)
        self.macro_f1.to(device)
        self.micro_f1.to(device)
        self.weigh_f1.to(device)

        self.cnfs_mat.to(device)

        self.mcc.to(device)

    def forward(self, x, y):
        self.categ_pc(x, y)
        self.macro_pc(x, y)
        self.micro_pc(x, y)
        self.weigh_pc(x, y)

        self.categ_rc(x, y)
        self.macro_rc(x, y)
        self.micro_rc(x, y)
        self.weigh_rc(x, y)

        self.categ_f1(x, y)
        self.macro_f1(x, y)
        self.micro_f1(x, y)
        self.weigh_f1(x, y)

        self.cnfs_mat(x, y)
        self.mcc(x, y)

In [8]:
class MultiLayerPerceptron(LightningModule):
	def __init__(self, input_dim, num_classes, labels):
		super().__init__()
		self.input_dim = input_dim
		self.num_classes = num_classes
		self.dense = nn.Linear(self.input_dim, self.input_dim)
		self.relu = nn.ReLU()
		self.linear = nn.Linear(self.input_dim, self.num_classes)
		self.model = nn.Sequential(self.dense, self.relu, self.linear)
		self.loss = nn.CrossEntropyLoss()
		self.labels = labels if labels is not None else list(range(self.num_classes))

	def setup(self, stage=None) -> None:
		if stage == 'fit':
			self.val_metrics = ClassificationMetrics(self.num_classes, "val")
		if stage == 'test':
			self.test_metrics = ClassificationMetrics(self.num_classes, "test")

	def forward(self, x):
		result = self.model(x)
		return result

	def configure_optimizers(self):
		return torch.optim.AdamW(self.parameters(), lr=1e-3)

	def training_step(self, batch, batch_idx):
		x, y = batch
		pred = self(x)
		y = y.type_as(pred).long()
		loss = self.loss(pred, y)
		self.log("train_loss", loss)
		return loss

	def validation_step(self, batch, batch_idx):
		x, y = batch
		pred = self(x)
		y = y.type_as(pred).long()
		loss = self.loss(pred, y)
		self.val_metrics(pred, y)
		self.log("val_loss", loss)
		self.log_scalars(self.val_metrics)

	def validation_epoch_end(self, outputs):
		self.log_nonscalars(self.val_metrics)

	def test_step(self, batch, batch_idx):
		x, y = batch
		pred = self(x)
		y = y.type_as(pred).long()
		loss = self.loss(pred, y)
		self.test_metrics(pred, y)
		self.log("test_loss", loss)
		self.log_scalars(self.test_metrics)

	def test_epoch_end(self, outputs):
		self.log_nonscalars(self.test_metrics)

	def log_scalars(self, metric: ClassificationMetrics):
		self.log(f"{metric.mode}_precision_macro", metric.macro_pc)
		self.log(f"{metric.mode}_precision_micro", metric.micro_pc)
		self.log(f"{metric.mode}_precision_weighted", metric.weigh_pc)

		self.log(f"{metric.mode}_recall_macro", metric.macro_rc)
		self.log(f"{metric.mode}_recall_micro", metric.micro_rc)
		self.log(f"{metric.mode}_recall_weighted", metric.weigh_rc)

		self.log(f"{metric.mode}_f1_macro", metric.macro_f1)
		self.log(f"{metric.mode}_f1_micro", metric.micro_f1)
		self.log(f"{metric.mode}_f1_weighted", metric.weigh_f1)

		self.log(f"{metric.mode}_mcc", metric.mcc)

	def log_nonscalars(self, metric: ClassificationMetrics):
		fig = plt.figure(figsize=(24, 24))
		cf_matrix = metric.cnfs_mat.compute().cpu().numpy()
		sb.heatmap(cf_matrix, annot=True, xticklabels=self.labels, yticklabels=self.labels, fmt='.1%')
		self.logger.experiment.add_figure(f"{metric.mode}_cnfs_mat", fig, global_step=self.current_epoch)

		categ_pc = metric.categ_pc.compute().cpu().tolist()
		pc_map = dict(zip(self.labels, categ_pc))
		self.logger.experiment.add_scalars(f"{metric.mode}_precision_categ", pc_map, global_step=self.current_epoch)

		categ_rc = metric.categ_pc.compute().cpu().tolist()
		rc_map = dict(zip(self.labels, categ_rc))
		self.logger.experiment.add_scalars(f"{metric.mode}_recall_categ", rc_map, global_step=self.current_epoch)

		categ_f1 = metric.categ_pc.compute().cpu().tolist()
		f1_map = dict(zip(self.labels, categ_f1))
		self.logger.experiment.add_scalars(f"{metric.mode}_f1_categ", f1_map, global_step=self.current_epoch)

In [9]:
model = MultiLayerPerceptron(561, 12, defined_labels)
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
trainer = Trainer(gpus=0, max_epochs=50, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
trainer.test(model, test_dataloaders=test_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name        | Type                  | Params
------------------------------------------------------
0 | dense       | Linear                | 315 K 
1 | relu        | ReLU                  | 0     
2 | linear      | Linear                | 6.7 K 
3 | model       | Sequential            | 322 K 
4 | loss        | CrossEntropyLoss      | 0     
5 | val_metrics | ClassificationMetrics | 0     
------------------------------------------------------
322 K     Trainable params
0         Non-trainable params
322 K     Total params
1.288     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 60 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)
/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 12 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)
/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does n

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:679: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(
/Users/georgehuang/GitRepo/olsen/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_macro': 0.8495360612869263,
 'test_f1_micro': 0.9335862994194031,
 'test_f1_weighted': 0.9332208633422852,
 'test_loss': 0.2791445851325989,
 'test_mcc': 0.9222668409347534,
 'test_precision_macro': 0.8662391304969788,
 'test_precision_micro': 0.9335863590240479,
 'test_precision_weighted': 0.9363933205604553,
 'test_recall_macro': 0.8385277390480042,
 'test_recall_micro': 0.9335863590240479,
 'test_recall_weighted': 0.9335862994194031}
--------------------------------------------------------------------------------


[{'test_loss': 0.2791445851325989,
  'test_precision_macro': 0.8662391304969788,
  'test_precision_micro': 0.9335863590240479,
  'test_precision_weighted': 0.9363933205604553,
  'test_recall_macro': 0.8385277390480042,
  'test_recall_micro': 0.9335863590240479,
  'test_recall_weighted': 0.9335862994194031,
  'test_f1_macro': 0.8495360612869263,
  'test_f1_micro': 0.9335862994194031,
  'test_f1_weighted': 0.9332208633422852,
  'test_mcc': 0.9222668409347534}]